# Opptaksområder 2018

In [14]:
aargang <- 2018

In [15]:
options(repr.matrix.max.rows=600, repr.matrix.max.cols=2000)

suppressPackageStartupMessages({ 
library(tidyverse)
library(readxl)
library(klassR)
library(sf)
library(leaflet)
        })

## Kodeliste for opptaksområder i spesialisthelsetjenesten (somatikk)

In [16]:
opptaksomrader_KLASS <- klassR::GetKlass(629, output_style = "wide", date = c(paste0(aargang+1, "-01-01"))) %>%
dplyr::rename(GRUNNKRETSNUMMER = code4, 
             GRUNNKRETS_NAVN = name4, 
             OPPTAK_NUMMER = code3, 
             OPPTAK = name3, 
             ORGNR_HF = code2, 
             NAVN_HF = name2, 
             ORGNR_RHF = code1, 
             NAVN_RHF = name1)

nrow(opptaksomrader_KLASS)

[1] 14180

In [17]:
opptaksomrader_KLASS %>%
dplyr::filter(substr(GRUNNKRETSNUMMER, 1, 1) == "0")

GRUNNKRETSNUMMER,GRUNNKRETS_NAVN,OPPTAK_NUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [18]:
opptaksomrader_KLASS %>%
dplyr::filter(GRUNNKRETSNUMMER == "03010101") 

GRUNNKRETSNUMMER,GRUNNKRETS_NAVN,OPPTAK_NUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


### Korrespondanse mellom t og t-1 fra KLASS

In [19]:
grunnkrets_KLASS <- klassR::GetKlass(1, date = c(paste0(aargang, "-01-01"), paste0(aargang+1, "-01-01")), correspond = TRUE) %>%
dplyr::rename(GRUNNKRETSNUMMER_T1 = sourceCode, 
             GRUNNKRETSNUMMER = targetCode, 
             targetName = targetName)

nrow(grunnkrets_KLASS)

# grunnkrets_KLASS <- grunnkrets_KLASS %>%
# dplyr::filter(!GRUNNKRETSNUMMER_T1 %in% c("18500111") |  !GRUNNKRETSNUMMER %in% c("18061610"), # Fjerner Kjerrvika fra Storå
#              !GRUNNKRETSNUMMER_T1 %in% c("18500109") |  !GRUNNKRETSNUMMER %in% c("18750211"),  # Fjerner Indre Tysfjord fra Kjerrvika
#              !GRUNNKRETSNUMMER_T1 %in% c("18500119") |  !GRUNNKRETSNUMMER %in% c("18750211")) # Fjerner Bjørntoppen fra Kjerrvika

nrow(grunnkrets_KLASS)

[1] 43

[1] 43

OK

In [20]:
grunnkrets_KLASS %>%
group_by(GRUNNKRETSNUMMER) %>% # grunnkrets_KLASS_T1_2
filter(n()>1)

GRUNNKRETSNUMMER_T1,sourceName,GRUNNKRETSNUMMER,targetName
<chr>,<chr>,<chr>,<chr>
14440104,Kjøs,14490114,Markane
14490113,Markene,14490114,Markane


In [21]:
grunnkrets_KLASS %>%
group_by(GRUNNKRETSNUMMER_T1) %>% # grunnkrets_KLASS_T1_2
filter(n()>1)

GRUNNKRETSNUMMER_T1,sourceName,GRUNNKRETSNUMMER,targetName
<chr>,<chr>,<chr>,<chr>
02190314,Voll 14,02190317,Voll 14 Nord
02190314,Voll 14,02190318,Voll 14 Sør
02190315,Voll 15,02190319,Voll 15 Nord
02190315,Voll 15,02190320,Voll 15 Sør
02190503,Hosle sør 03,02190530,Hosle sør 03 Nord
02190503,Hosle sør 03,02190531,Hosle sør 03 Sør
02190504,Hosle sør 04,02190532,Hosle sør 04 Nord
02190504,Hosle sør 04,02190533,Hosle sør 04 Sør
02190508,Hosle sør 08,02190534,Hosle sør 08 Nord


### Merger opptaksområder med KLASS (t-1)

In [22]:
opptaksomrader_KLASS_2 <- opptaksomrader_KLASS %>%
dplyr::left_join(grunnkrets_KLASS, by = "GRUNNKRETSNUMMER") %>%
dplyr::mutate(GRUNNKRETSNUMMER_T1 = case_when(
is.na(GRUNNKRETSNUMMER_T1) ~ GRUNNKRETSNUMMER, 
    TRUE ~ GRUNNKRETSNUMMER_T1
)) %>%
dplyr::select(GRUNNKRETSNUMMER, GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF) %>%
dplyr::distinct(GRUNNKRETSNUMMER_T1, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF)

nrow(opptaksomrader_KLASS_2)

[1] 14181

In [23]:
# opptaksomrader_KLASS_2 %>%
# dplyr::filter(GRUNNKRETSNUMMER_T1 == "18500109")

### Laster inn KLASS (t-1)

In [24]:
grunnkrets_KLASS_T1 <- klassR::GetKlass(1, output_style = "wide", date = c(paste0(aargang, "-01-01"))) %>%
dplyr::rename(GRUNNKRETSNUMMER_T1 = code2, 
             GRUNNKRETS_NAVN_T1 = name2) %>%
dplyr::select(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1) 

nrow(grunnkrets_KLASS_T1)
nrow(opptaksomrader_KLASS_2)

[1] 14053

[1] 14181

In [25]:
# opptaksomrader_KLASS_2 %>%
# group_by(GRUNNKRETSNUMMER_T1) %>% # grunnkrets_KLASS_T1_2
# filter(n()>1)

# OBS: 
+ sjekk opp Kjøs/Markene
+ Vegusdal nord/Hovland

In [26]:
grunnkrets_KLASS_T1_2 <- grunnkrets_KLASS_T1 %>%
dplyr::left_join(opptaksomrader_KLASS_2, by = "GRUNNKRETSNUMMER_T1") %>%
dplyr::filter(!is.na(OPPTAK))

nrow(grunnkrets_KLASS_T1_2)

head(grunnkrets_KLASS_T1_2)

# grunnkrets_KLASS_T1_2 <- grunnkrets_KLASS_T1 %>%
# dplyr::distinct(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1, OPPTAK)

[1] 7417

,GRUNNKRETSNUMMER_T1,GRUNNKRETS_NAVN_T1,OPPTAK_NUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,10010101,Kjære-andås,19,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
2,10010102,Mæbø-høyfjellet,19,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
3,10010103,Lindebø-skålevik,19,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
4,10010201,Andøya,19,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
5,10010202,Voie,19,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
6,10010203,Møvik,19,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF


### Lagrer filen 

In [301]:
# test <- grunnkrets_KLASS_T1_2 %>%
#   dplyr::mutate(TOM_FORELDER = "") %>%
#   # dplyr::filter(opptak %in% c("Stavanger", "Ålesund")) %>%
#   dplyr::select(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1, OPPTAK_NUMMER, ORGNR_HF, NAVN_HF, OPPTAK, ORGNR_RHF, NAVN_RHF, TOM_FORELDER)

# level_1 <- test %>%
#   dplyr::select(ORGNR_RHF, TOM_FORELDER, NAVN_RHF) %>%
#   dplyr::rename('ns1:kode' = ORGNR_RHF, 
#                 'ns1:forelder' = TOM_FORELDER, 
#                 'ns1:navn_bokmål' = NAVN_RHF) %>%
#   dplyr::distinct()

# level_2 <- test %>%
#   dplyr::select(ORGNR_HF, ORGNR_RHF, NAVN_HF) %>%
#   dplyr::rename('ns1:kode' = ORGNR_HF, 
#                 'ns1:forelder' = ORGNR_RHF, 
#                 'ns1:navn_bokmål' = NAVN_HF) %>%
#   dplyr::distinct()

# level_3 <- test %>%
#   dplyr::select(OPPTAK_NUMMER, ORGNR_HF, OPPTAK) %>%
#   dplyr::rename('ns1:kode' = OPPTAK_NUMMER, 
#                 'ns1:forelder' = ORGNR_HF, 
#                 'ns1:navn_bokmål' = OPPTAK) %>%
#   dplyr::distinct()

# level_4 <- test %>%
#   dplyr::select(GRUNNKRETSNUMMER_T1, OPPTAK_NUMMER, GRUNNKRETS_NAVN_T1) %>%
#   dplyr::rename('ns1:kode' = GRUNNKRETSNUMMER_T1, 
#                 'ns1:forelder' = OPPTAK_NUMMER, 
#                 'ns1:navn_bokmål' = GRUNNKRETS_NAVN_T1) %>%
#   dplyr::distinct()

# KLASS <- rbind(level_1, level_2, level_3, level_4)

In [302]:
# openxlsx::write.xlsx(KLASS, file = paste0("/ssb/bruker/rdn/opptak_", aargang, "2019.xlsx"),
#                      rowNames = FALSE,
#                      showNA = FALSE,
#                      overwrite=T) # T = overskriver dersom filen allerede finnes, F = gir feilmelding dersom filen finnes